# Playground for Multihead Attention

## Prototype Attention Head
Implementation of a single attention head (Just for demonstration of attention principle. Not used later)

In [5]:
import torch
import torch.nn as nn

torch.set_printoptions(threshold=10, edgeitems=3)
torch.manual_seed(42)

## This is just for demonstration purposes. Not used later
class CausalSelfAttention(nn.Module):

    # input embedding dimension must not be same as output embedding dimension
    # context length sets number of tokens in input 
    # dropout rate is a value for randomly setting weights to 0 (prevents overfitting)
    # qkv_bias ads learnable bias weights to nn.Linear. Not used in GPT2 no significant improvement
    def __init__(self, input_dim, output_dim, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.output_dim = output_dim
        
        # init weight matrices that are later used for input -> query, key, value projection
        # all projections (every token) are done with these same matrices
        self.W_query = nn.Linear(input_dim, output_dim, bias=qkv_bias)
        self.W_key = nn.Linear(input_dim, output_dim, bias=qkv_bias)
        self.W_value = nn.Linear(input_dim, output_dim, bias=qkv_bias)
        
        # init dropout (prevent overfitting)
        self.dropout = nn.Dropout(dropout) # dropout is a probability
        
        # registers static causal masking matrix (diagonal) in same device as model (is self.mask)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # Causal Masking on diagonal triangle

    def forward(self, x):
        
        # stores dimensions of input (here only context length needed)
        batch_size, con_len, in_dim = x.shape # set them to the shape of x ( 8, 4, 256 )
        
        # using the weight matrices for projection of the input to query, key and value
        # broadcasting (for badges)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # calculates all attention scores
        # Compares query for every input token to all keys
        attn_scores = queries @ keys.transpose(1, 2)

        # creates bool mask (diagonal) fills all causal invisible elements with -inf
        attn_scores.masked_fill( self.mask.bool()[:con_len, :con_len], -torch.inf)

        # calculations of soft max scaled with sqrt of keys length dimension (keys.shape)
        # note: dim=-1 takes most internal dimension in an array 
        # example: [ [ [ 1, 2, 3] ],  [ [ 4, 5, 6 ], [ 7, 8, 8 ] ] ] -> 3
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # applying dropout to weights
        # randomly sets weights to 0 based on probability
        # !! scales remaining values up to compensate dropped weights
        attn_weights = self.dropout(attn_weights) # Additional random dropout

        # sum of attention weighted values for every input (including causal masked knowledge)
        context_vec = attn_weights @ values
        return context_vec


## Short insight in masking and causal masking

In [6]:
# Triangular Matrix filtering example for causal masking
def mask_demo():

    # matrix to mask. Example attention scores
    matrix = torch.rand(4,4)

    # buffered static causal mask
    mask = torch.triu(torch.ones(4, 4), diagonal=1)

    print("\nExample matrix with random attention_scores:\n", matrix)
    print("\nBuffered causal (autoregressive) mask:\n", mask)  # Triangle mask: 0s below diagonal, 1s above
    print("\nmask.bool():\n", mask.bool())  # Converts to boolean: 0→False, 1→True
    print("\nmask.bool() clipped to match input dimension (context):\n", mask.bool()[:3, :3])  # 3x3 sub-matrix
    print("\nmatrix.masked_fill(mask.bool(), -inf):\n", matrix.masked_fill(mask.bool()[:4, :4], -torch.inf))  # Sets upper triangle to -inf

if '__file__' not in dir(): mask_demo()


Example matrix with random attention_scores:
 tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936],
        [0.8694, 0.5677, 0.7411, 0.4294]])

Buffered causal (autoregressive) mask:
 tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])

mask.bool():
 tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

mask.bool() clipped to match input dimension (context):
 tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

matrix.masked_fill(mask.bool(), -inf):
 tensor([[0.8823,   -inf,   -inf,   -inf],
        [0.3904, 0.6009,   -inf,   -inf],
        [0.9408, 0.1332, 0.9346,   -inf],
        [0.8694, 0.5677, 0.7411, 0.4294]])


## Multi Head Attention
An implementation of Multi Head Attention.
Multiple heads train themselves on different parts of the input embedding. The input is projected to Q, K, V matrices (all at once), then split into "head"-sized slices.
Each head computes attention scores (distance between Q and K) (Q @ K^T) and attention weights (softmax) independently on its slice.
The resulting context vectors from all heads are concatenated and projected through a final linear layer.

In [7]:
class MultiHeadAttention(nn.Module):

    def __init__(self, input_dim, output_dim, context_length, dropout, num_heads, qkv_bias=False, verbose=False):
        super().__init__()
        
        # Check if output dimension is dividable by attention head number without rest
        # Input is split to num_head chunks of length head_dim
        assert output_dim % num_heads == 0, "Output dimension must be dividable by head_num"

        self.TransformerBlock = output_dim
        self.num_heads = num_heads
        self.head_dim = output_dim // num_heads

        # Init weight matrices for input to qkv projection (full not yet separated into head parts)
        self.W_query = nn.Linear(input_dim, output_dim, qkv_bias)
        self.W_key = nn.Linear(input_dim, output_dim, qkv_bias)
        self.W_value = nn.Linear(input_dim, output_dim, qkv_bias)

        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
        self.dropout = nn.Dropout(dropout)
        
        self.out_proj = nn.Linear(output_dim, output_dim) # Linear layer for combination of head outputs
        
        if verbose:
            print(f"\n=== MultiHeadAttention Initialization ===")
            print(f"    input_dim =", input_dim)
            print(f"    output_dim =", output_dim)
            print(f"    num_heads =", self.num_heads)
            print(f"    head_dim =", self.head_dim)
            print(f"    Generating nn.Linear({input_dim}, {output_dim}) weights for query, key and value")
            print(f"    Generating causal diagonal mask torch.triu(torch.ones({context_length}, {context_length}), diagonal=1) for causal masking of attn_scores")        
            print(f"    Generating dropout nn.Dropout({dropout}) for random dropout of attn_weights")        
            print(f"    Generating optional nn.Linear({output_dim}, {output_dim}) weights for final context_vector projection")
            print(f"=== End MultiHeadAttention Initialization ===\n")
        
    def forward(self, x, verbose=False):

        # local variables for input shape
        batch_size, context_length, input_dim = x.shape

        if verbose:
            print(f"\n=== MultiHeadAttention Forward Pass ===")
            print(f"Input shape: {x.shape} (batch_size={batch_size}, context_length={context_length}, input_dim={input_dim})")
            print(f"Config: num_heads={self.num_heads}, head_dim={self.head_dim}, output_dim={self.num_heads * self.head_dim}")
            print(f"\nInput tensor (batch 0 with shape {x[0].shape}):")
            print(f" {x[0]}")

        # using weight matrices for projection of the input to qkv (not yet splitted for attention heads)
        # broadcasting for badges
        # -> shape: batch_size, context_length, output_dim
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        if verbose:
            print(f"\n1. QKV Projection:")
            print(f"   QKV shapes: {queries.shape}")
 
        # Implicitly splitting matrix by adding head_num dimension
        # Unrol last dim: (batch_size, context_length, output_dim) -> (batch_size, context_length, head_num, head_dim)
        # Example ( 8, 4, 256 ) -> ( 8, 4, 8, 32 ) for num_heads = 8 and head_dim = 32
        queries = queries.view(batch_size, context_length, self.num_heads, self.head_dim)
        keys = keys.view(batch_size, context_length, self.num_heads, self.head_dim)
        values = values.view(batch_size, context_length, self.num_heads, self.head_dim)

        if verbose:
            print(f"\n2. Split into heads:")
            print(f"(batch_size, context_length, output_dim) -> (batch_size, context_length, head_num, head_dim)")
            print(f"   QKV shapes after view: {queries.shape}")
 
        # Transpose to use for query comparison - move num_head to front
        # (batch_size, context_length, num_heads, head_dim) -> (batch_size, num_head, context_length, head_dim)
        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)

        if verbose:
            print(f"\n3. Transpose for attention computation:")
            print(f"(batch_size, context_length, num_heads, head_dim) -> (batch_size, num_head, context_length, head_dim)")
            print(f"   QKV shapes after transpose: {queries.shape}")

        # Compute scaled dot_production attention
        attn_scores = queries @ keys.transpose(2,3) # Dot product for each head

        if verbose:
            print(f"\n4. Attention scores computation:")
            print(f"   attn_scores shape: {attn_scores.shape}")
            print(f"   Scale factor (1/sqrt(head_dim)): {1/keys.shape[-1]**0.5:.4f}")
            print(f"   Raw attention scores for head 0, batch 0:\n{attn_scores[0, 0]}")

        # Causal Masking
        mask_bool = self.mask.bool()[:context_length, :context_length]
        attn_scores = attn_scores.masked_fill(mask_bool, -torch.inf)

        if verbose:
            print(f"\n5. Causal masking:")
            print(f"   Causal mask:\n{mask_bool}\n")
            print(f"   Then masked_fill True -> -torch.inf\n")
            print(f"   Masked attention scores for head 0, batch 0:\n{attn_scores[0, 0]}")

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim =-1)

        if verbose:
            print(f"\n6. Softmax attention weights:")
            print(f"   attn_weights shape: {attn_weights.shape}")
            print(f"   Attention weights for head 0, batch 0:\n{attn_weights[0, 0]}\n")
            print(f"   Sum of weights (should be ~1.0): {attn_weights[0, 0].sum(dim=-1)}")

        attn_weights = self.dropout(attn_weights)

        if verbose:
            print(f"\n7. After dropout:")
            print(f"   Attention weights after dropout for head 0, batch 0:\n{attn_weights[0, 0]}")

        # build context vector switch back num_heads and context_length, combining heads
        # self.output_dim = self.num_heads * self.head_dim
        context_vec = (attn_weights @ values).transpose(1,2)

        if verbose:
            print(f"\n8. Compute context vectors:")
            print(f"   context_vec shape after attention: {context_vec.shape}")
            print(f"   First context vector (batch 0, token 0, head 0): {context_vec[0, 0, 0]}...")

        context_vec = context_vec.contiguous().view(batch_size, context_length, self.num_heads * self.head_dim)

        if verbose:
            print(f"\n9. Concatenate heads:")
            print(f"   context_vec shape after view: {context_vec.shape}")
            print(f"   First concatenated context vector (batch 0): {context_vec[0]}...")

        context_vec = self.out_proj(context_vec) # optional projection by Linear layer

        if verbose:
            print(f"\n10. Final output projection:")
            print(f"   Final context_vec shape: {context_vec.shape}")
            print(f"   Final context vector (batch 0): {context_vec[0]}...")
            print(f"=== End MultiHeadAttention Forward Pass ===\n")

        return context_vec

## Test Run

In [8]:
def multi_head_attention_test_run(verbose=False):
    if verbose: print("\n\n------- initializing Multi Head Attention ----------------\n")
    mha = MultiHeadAttention(input_dim=768, output_dim=128, context_length=4, dropout=0.2, num_heads=8, verbose=verbose)

    if verbose: print("\n\n----- Using real data input (see DataPreparation.ipynb) --\n")
    %run "./01. DataPreparation.ipynb"
    batch = get_test_input_embedding(verbose=verbose) # defined in DataPreparation.ipynb

    if verbose: print("\n\n------- performing multi head attention ------------------\n")
    context_vector = mha(batch, verbose=verbose)

if '__file__' not in dir(): _test_run = multi_head_attention_test_run(True)



------- initializing Multi Head Attention ----------------


=== MultiHeadAttention Initialization ===
    input_dim = 768
    output_dim = 128
    num_heads = 8
    head_dim = 16
    Generating nn.Linear(768, 128) weights for query, key and value
    Generating causal diagonal mask torch.triu(torch.ones(4, 4), diagonal=1) for causal masking of attn_scores
    Generating dropout nn.Dropout(0.2) for random dropout of attn_weights
    Generating optional nn.Linear(128, 128) weights for final context_vector projection
=== End MultiHeadAttention Initialization ===



----- Using real data input (see DataPreparation.ipynb) --


=== Embedder Initialization ===
    vocab_size =  50252
    context_length =  4
    embedding_dim =  768
    Generating token_embeddings (50252 x 768)
    Generating pos_embeddings (4 x 768)
=== End Initialization ===

Displaying first row of batch

First batch elements Input x:
 tensor([ 632, 3160,  287,  262])  <-->   It lives in the

First batch elements Target 